In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Krok 1: Zdefiniuj modele Pydantic dla uporządkowanych wyników

Te modele definiują **schemat**, który agenci będą zwracać. Korzystanie z `response_format` z Pydantic zapewnia:
- ✅ Bezpieczne typowanie danych
- ✅ Automatyczną walidację
- ✅ Brak błędów parsowania w odpowiedziach w formie wolnego tekstu
- ✅ Łatwe warunkowe kierowanie na podstawie pól


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Krok 2: Utwórz narzędzie do rezerwacji hotelu

To narzędzie będzie używane przez **availability_agent**, aby sprawdzić dostępność pokoi. Używamy dekoratora `@ai_function`, aby:
- Przekształcić funkcję w Pythonie w narzędzie wywoływane przez AI
- Automatycznie wygenerować schemat JSON dla LLM
- Obsłużyć walidację parametrów
- Umożliwić automatyczne wywoływanie przez agentów

W tym demo:
- **Sztokholm, Seattle, Tokio, Londyn, Amsterdam** → Pokoje dostępne ✅
- **Wszystkie inne miasta** → Brak pokoi ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Krok 3: Zdefiniuj funkcje warunkowe dla routingu

Te funkcje analizują odpowiedź agenta i określają, którą ścieżkę wybrać w przepływie pracy.

**Kluczowy schemat:**
1. Sprawdź, czy wiadomość to `AgentExecutorResponse`
2. Przeanalizuj strukturalny wynik (model Pydantic)
3. Zwróć `True` lub `False`, aby kontrolować routing

Przepływ pracy oceni te warunki na **krawędziach**, aby zdecydować, który executor wywołać jako następny.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Krok 4: Utwórz niestandardowy wykonawca wyświetlania

Wykonawcy to komponenty przepływu pracy, które wykonują transformacje lub efekty uboczne. Używamy dekoratora `@executor`, aby stworzyć niestandardowego wykonawcę, który wyświetla ostateczny wynik.

**Kluczowe pojęcia:**
- `@executor(id="...")` - Rejestruje funkcję jako wykonawcę przepływu pracy
- `WorkflowContext[Never, str]` - Podpowiedzi typów dla wejścia/wyjścia
- `ctx.yield_output(...)` - Zwraca ostateczny wynik przepływu pracy


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Krok 5: Załaduj zmienne środowiskowe

Skonfiguruj klienta LLM. Ten przykład działa z:
- **Modelami GitHub** (Bezpłatny poziom z tokenem GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Krok 6: Tworzenie agentów AI z uporządkowanymi wynikami

Tworzymy **trzy wyspecjalizowane agenty**, każdy opakowany w `AgentExecutor`:

1. **availability_agent** - Sprawdza dostępność hoteli za pomocą narzędzia
2. **alternative_agent** - Proponuje alternatywne miasta (gdy brak wolnych pokoi)
3. **booking_agent** - Zachęca do rezerwacji (gdy pokoje są dostępne)

**Kluczowe funkcje:**
- `tools=[hotel_booking]` - Udostępnia narzędzie agentowi
- `response_format=PydanticModel` - Wymusza uporządkowany format JSON
- `AgentExecutor(..., id="...")` - Opakowuje agenta do użycia w przepływie pracy


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Krok 7: Budowanie przepływu pracy z krawędziami warunkowymi

Teraz użyjemy `WorkflowBuilder`, aby skonstruować graf z warunkowym trasowaniem:

**Struktura przepływu pracy:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Kluczowe metody:**
- `.set_start_executor(...)` - Ustawia punkt wejścia
- `.add_edge(from, to, condition=...)` - Dodaje krawędź warunkową
- `.build()` - Finalizuje przepływ pracy


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Krok 8: Uruchom przypadek testowy 1 - Miasto BEZ dostępności (Paryż)

Przetestujmy ścieżkę **braku dostępności**, prosząc o hotele w Paryżu (który w naszej symulacji nie ma dostępnych pokoi).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Krok 9: Uruchom Test Case 2 - Miasto Z Dostępnością (Sztokholm)

Teraz przetestujmy ścieżkę **dostępności**, wysyłając zapytanie o hotele w Sztokholmie (który w naszej symulacji ma dostępne pokoje).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Kluczowe informacje i kolejne kroki

### ✅ Czego się nauczyłeś:

1. **Wzorzec WorkflowBuilder**
   - Użyj `.set_start_executor()`, aby zdefiniować punkt wejścia
   - Użyj `.add_edge(from, to, condition=...)` do warunkowego routingu
   - Wywołaj `.build()`, aby sfinalizować przepływ pracy

2. **Warunkowy routing**
   - Funkcje warunkowe analizują `AgentExecutorResponse`
   - Przetwarzaj strukturalne dane wyjściowe, aby podejmować decyzje o routingu
   - Zwróć `True`, aby aktywować krawędź, `False`, aby ją pominąć

3. **Integracja narzędzi**
   - Użyj `@ai_function`, aby przekształcić funkcje Pythona w narzędzia AI
   - Agenci automatycznie wywołują narzędzia, gdy jest to potrzebne
   - Narzędzia zwracają JSON, który agenci mogą analizować

4. **Strukturalne dane wyjściowe**
   - Użyj modeli Pydantic do bezpiecznego typowo wyodrębniania danych
   - Ustaw `response_format=MyModel` podczas tworzenia agentów
   - Analizuj odpowiedzi za pomocą `Model.model_validate_json()`

5. **Niestandardowe wykonawcy**
   - Użyj `@executor(id="...")`, aby tworzyć komponenty przepływu pracy
   - Wykonawcy mogą przekształcać dane lub wykonywać efekty uboczne
   - Użyj `ctx.yield_output()`, aby generować wyniki przepływu pracy

### 🚀 Zastosowania w rzeczywistości:

- **Rezerwacja podróży**: Sprawdzanie dostępności, sugerowanie alternatyw, porównywanie opcji
- **Obsługa klienta**: Routing na podstawie rodzaju problemu, nastroju, priorytetu
- **E-commerce**: Sprawdzanie stanów magazynowych, sugerowanie alternatyw, realizacja zamówień
- **Moderacja treści**: Routing na podstawie wyników toksyczności, zgłoszeń użytkowników
- **Przepływy zatwierdzeń**: Routing na podstawie kwoty, roli użytkownika, poziomu ryzyka
- **Przetwarzanie wieloetapowe**: Routing na podstawie jakości danych, kompletności

### 📚 Kolejne kroki:

- Dodaj bardziej złożone warunki (wielokrotne kryteria)
- Zaimplementuj pętle z zarządzaniem stanem przepływu pracy
- Dodaj podprzepływy dla komponentów wielokrotnego użytku
- Zintegruj z rzeczywistymi API (rezerwacja hoteli, systemy magazynowe)
- Dodaj obsługę błędów i ścieżki awaryjne
- Wizualizuj przepływy pracy za pomocą wbudowanych narzędzi do wizualizacji



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
